This notebook will document a data analysis of the MOMA alternative data transformed to Linked Art JSON-LD.

The .json data files are in the data directory as follows:

- data
  - activity
  - group
  - person


activity - exhibition event 
group - organisation involved in exhibition events
person - person involved in exhibition events


# Activity - data analysis

In [1]:
try:
    import json
except:
    %pip install json
    import json

try:
    import requests
except:
    %pip install requests
    import requests

try:
    import pandas as pd
except:
    %pip install pandas
    import pandas as pd

In [2]:
from operator import itemgetter

filepath = "data/activity/"

exhibitions = []

exhibition_template = {
    "id": "",
    "label": "",
    "start": "",
    "end": "",
    "location": "",
    "org": "",
    "_label":"",
    "identified_by": [],
    "timespan": [],
    "carried_out_by": [],
    "influenced_by": []
}


for x in range(1,54829,1):
    try:
        fpath = filepath + str(x) + ".json"
        
        with open(fpath) as json_file:
            data = json.load(json_file)

            this_exhibition = exhibition_template.copy()
            this_exhibition["id"] = data.get("id")
            this_exhibition["label"] = data.get("_label")
            this_exhibition["_label"] = data.get("_label")

            this_exhibition["start"] = data.get("timespan").get("begin_of_the_begin")
            this_exhibition["end"] = data.get("timespan").get("end_of_the_end")

            this_exhibition["location"] = data.get("took_place_at")[0].get("defined_by")


            this_exhibition["carried_out_by"] = [data.get("carried_out_by")[0].get("id")]

            influencers = []
        
            for inf in data["influenced_by"]:
                influencers.append(inf.get("id"))

            this_exhibition["influenced_by"] = influencers

            exhibition_copy = this_exhibition.copy()
            exhibitions.append(exhibition_copy) 
            
    except:
        continue

# order list by start date desc
sorted_list_start_desc = sorted(exhibitions, key=itemgetter('start'), reverse=True) 
for idx, entry in enumerate(sorted_list_start_desc):
        if entry.get('born') == "":
                sorted_list_start_desc.pop(idx)
        # get first 200 records
        sliced_list_born_desc = sorted_list_born_desc[:200]
        # write to file 
        with open('api/persons_born_desc.json', 'w') as file:
            file.write(json.dumps({"persons": sliced_list_born_desc}))

with open('api/events_all.json', 'w') as ex_file:
     ex_file.write(json.dumps({"events": exhibitions}))


NameError: name 'sorted_list_born_desc' is not defined

In [ ]:
from operator import itemgetter

with open('api/persons_all.json', 'r') as json_file:
    data = json.load(json_file)
    

for idx, entry in enumerate(data):
            if entry['start'] == "":
                data.pop(idx)
            elif entry['end'] == "":
                data.pop(idx)
            elif entry['_label'] == "":
                data.pop(idx)


# order by start date

# order by end date

# order by exhibition title

# save moma org  and non-moma records in separate files



In [3]:
filepath = "data/person/"

persons = []

person_template = {
    "id": "",
    "label": "",
    "_label":"",
    "name":"",
    "born": "",
    "died": "",
    "nationality": "",
    "identified_by": [],
    "referred_to_by": [],
    "equivalent": [],
    "total_exhibitions": [],
}

  

for x in range(1,73707,1):
    try:
        fpath = filepath + str(x) + ".json"
        
        with open(fpath) as json_file:
            data = json.load(json_file)
        
            this = person_template.copy()
            this["id"] = data.get("id")
            this["label"] = data.get("_label")
            this["_label"] = data.get("_label")

            this["name"] = data.get("identified_by")[0].get("content")

            if "born" in data:
                this["born"] = data.get("born").get("timespan").get("begin_of_the_begin")
            if "died" in data:
                this["died"] = data.get("died").get("timespan").get("end_of_the_end")

            copy = this.copy()
            persons.append(copy) 
            
    except:
        continue

with open('api/persons_all.json', 'w') as file:
     file.write(json.dumps({"persons": persons}))





# Add statistics to person summary records

- total number of exhibitions artist has featured in, in dataset

In [ ]:
from operator import itemgetter
# importing datetime
from datetime import datetime

with open('api/events_all.json', 'r') as exjson_file:
    exdata = json.load(exjson_file)

    # get list of lists of person ids = artists
    all_artists = [d['influenced_by'] for d in exdata.get('events')]

    # dictionary of exhibition id and artist list
    exhibitions = {}
    for ex in exdata.get('events'):
        idx = ex.get("id")
        exhibitions[idx] = {"_label": ex.get("_label") , "persons":ex.get('influenced_by')}


    
    # flatten list of lists to single list
    all_ex_list =  [item for sublist in all_artists for item in sublist]
    
updated_persons_list = []

with open('api/persons_all.json', 'r') as json_file:
    data = json.load(json_file)
    for person in data["persons"]:
        person_id = person.get("id")

        person_exhibitions = []
        for id in exhibitions:
            if person_id in exhibitions[id].get("persons"):
                exhibition = exhibitions[id]
                print(exhibition)
                label = exhibition.get("_label")
                start_date = exhibition.get("start_date")
                person_exhibitions.append({"id":id,"_label": label, "start_date": start_date})
        
        total_exhibitions = all_ex_list.count(person_id)

        updated_person = person.copy()
        updated_person["total_exhibitions"] = total_exhibitions
        updated_person["exhibitions"] = person_exhibitions
         

        updated_persons_list.append(updated_person)

        for idx, entry in enumerate(updated_persons_list):
            if entry['born'] == "":
                updated_persons_list.pop(idx)
            elif entry['died'] == "":
                updated_persons_list.pop(idx)
            elif entry['name'] == "":
                updated_persons_list.pop(idx)

        # order list by total exhibitions
        sorted_list_total_ex_desc = sorted(updated_persons_list, key=itemgetter('total_exhibitions'), reverse=True) 
        # get first 200 records
        sliced_list_total_ex_desc = sorted_list_total_ex_desc[:200]
        # write to file 
        with open('api/persons_total_exhibitions_desc.json', 'w') as file:
            file.write(json.dumps({"persons": sliced_list_total_ex_desc}))

        # order list by birth date desc


        sorted_list_born_desc = sorted(updated_persons_list, key=itemgetter('born'), reverse=True) 


        # get first 200 records
        sliced_list_born_desc = sorted_list_born_desc[:200]
        # write to file 
        with open('api/persons_born_desc.json', 'w') as file:
            file.write(json.dumps({"persons": sliced_list_born_desc}))
        
        # order list by birth date asc
        sorted_list_born_asc = sorted(updated_persons_list, key=itemgetter('born'), reverse=False) 
        # get first 200 records
        sliced_list_born_asc = sorted_list_born_asc[:200]
        # write to file 
        with open('api/persons_born_asc.json', 'w') as file:
            file.write(json.dumps({"persons": sliced_list_born_asc}))

        # order list by death date desc
        sorted_list_died_desc = sorted(updated_persons_list, key=itemgetter('died'), reverse=True) 
        # get first 200 records
        sliced_list_died_desc = sorted_list_died_desc[:200]
        # write to file 
        with open('api/persons_died_desc.json', 'w') as file:
            file.write(json.dumps({"persons": sliced_list_died_desc}))

        # order list by death date asc
        sorted_list_died_asc = sorted(updated_persons_list, key=itemgetter('died'), reverse=False) 
        # get first 200 records
        sliced_list_died_asc = sorted_list_died_asc[:200]
        # write to file 
        with open('api/persons_died_asc.json', 'w') as file:
            file.write(json.dumps({"persons": sliced_list_died_asc}))

        # order list by name desc
        sorted_list_name_desc = sorted(updated_persons_list, key=itemgetter('name'), reverse=True)
        # get first 200 records
        sliced_list_name_desc = sorted_list_name_desc[:200]
        # write to file 
        with open('api/persons_name_desc.json', 'w') as file:
            file.write(json.dumps({"persons": sliced_list_name_desc}))

        # order list by name asc
        sorted_list_name_asc = sorted(updated_persons_list, key=itemgetter('name'), reverse=False) 
        # get first 200 records
        sliced_list_name_asc = sorted_list_name_asc[:200]
        # write to file 
        with open('api/persons_name_desc.json', 'w') as file:
            file.write(json.dumps({"persons": sliced_list_name_asc}))
       
    